In [8]:
import os
import cv2
import numpy as np
# from modules.slide import Slide
# from modules.dsa import DSAFolder, DSAItem, DSA
import json

import glob
def generateAnno(contoursColors):

    annos = []
    elems = []

    for q in range(len(contoursColors)):
        cnts = contoursColors[q]['cnt']
        color = contoursColors[q]['color']

        for f in range(len(cnts)):
            cnt = cnts[f][0]
            hie = cnts[f][1]
            # print(hie)
            if hie[3]==-1:
                points = []
                for i in cnt:
                    for j in i:
                        x = int(j[0])
                        y = int(j[1]) 
                        points.append([x, y, 0])

                elem = {
                    "type": "polyline",
                    "lineColor": f"rgb({color})", 
                    "lineWidth": 3, 
                    "fillColor": f"rgba({color}, 0.5)",
                    "points": None,
                    "closed": True
                }

                elem["points"] = points

                if len(points)>1:
                    if (f+1)<len(cnts) and cnts[f+1][1][3]==-1: #No Children
                        pass
                    else: # Get Children
                        z = f+1
                        holes = []
                        while True:
                            hole = []
                            try:
                                cntChild = cnts[z][0]
                                hieChild = cnts[z][1]
                            except:
                                break
                            if hieChild[3]==-1:
                                break
                            for i in cntChild:
                                for j in i:
                                    x = int(j[0]) 
                                    y = int(j[1]) 
                                    hole.append([x, y, 0])
                            holes.append(hole)
                            z+=1
                        elem["holes"] = holes
                    elems.append(elem)

    anno = {
        "name": 'tbm_ai_filtered', 
        "description": 'tbm_ai_filtered',  
        "elements": None                        
    }
    anno["elements"] = elems
    annos.append(anno)

    return annos



def getContours(mask):

    def smoothBinary(mask2d):
        kn = 2
        iterat = 2
        kernel = np.ones((kn, kn), np.uint8) 
        for _ in range(iterat):
            mask2d = cv2.erode(mask2d, kernel, iterations=1) 
            mask2d = cv2.dilate(mask2d, kernel, iterations=1) 
        return mask2d
    
    mask2 = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    
    maskSmooth = smoothBinary(mask2)

    thresholdArea = 0.0001

    contours, hierarchy = cv2.findContours(maskSmooth, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    print(f"found {len(contours)} contours")

    c_max = max(contours, key = cv2.contourArea)
    area_min = cv2.contourArea(c_max) * thresholdArea
    
    print(len(contours))
    
    cntheis = sorted(zip(contours, hierarchy[0]), key=lambda b: cv2.contourArea(b[0]) , reverse=True)
    
    mxmx = int(1.0 * len(contours))
    
    contours_filtered = []
    
    for qq in range(mxmx):
        if qq%50==0:
            print(f"processing {qq}/{mxmx}")
        contour = cntheis[qq][0]
        hie = cntheis[qq][1]
        contours_filtered.append((contour,hie))

    return contours_filtered

xx = 35873
yy = 58561

mask = np.zeros((yy, xx, 3), dtype='uint8')

# mydir = "/blue/pinaki.sarder/manojkumargalla/filtered_image_tiles/filtered_image/"
tiles_folder = "/blue/pinaki.sarder/manojkumargalla/PostProcess/data/WSI_tiles_predictions/"

filenames = glob.glob(f'{tiles_folder}/*/')
for filename in filenames:
    files = os.listdir(f'{filename}')
    print(f'processing file - {filename}')
    xx = int(files[0].split('_')[2])
    yy = int(files[0].split('_')[3])
    mask = np.zeros((yy, xx, 3), dtype='uint8')
    for file in files:
        # lst = file.split('[')[-1].split(']')[0].replace("=", "").replace("x", "").replace("w", "").replace("y", "").replace("h", "").split(",")
        # print(lst)
        print(file)
        x = int(file.split('_')[4].strip('x'))
        # x, y, w, h = [int(k) for k in lst]
        y = int(file.split('_')[5].split('.')[0].strip('y'))
        w, h = 512, 512
        # mask[y:y+h, x:x+w, :] = cv2.imread( f"{mydir}/{file}",3)
        img = cv2.imread( f"{filename}/{file}",3)
        h, w, _ = img.shape
        if y + h <= yy and x + w <= xx:
            mask[y:y+h, x:x+w, :] = img
        else:
            print(f"Skipping tile {file} due to out-of-bounds placement.")
            print(f'y + h is {y+h}, x + w is {x+w}')

    contours = getContours(mask)
    contoursColors = [{'cnt': contours, 'color': '255, 20, 20'}]

    # dsafid = {
    #     '32-2_PAS_1of2':'6691a0ba0519743b266f59c8'
    # }

    # slides2 = [
    #     '32-2_PAS_1of2'
    # ]

    # annoooos = {}

    # for name in slides2:
    #     config = {
    #         'fid': dsafid[name],
    #         'username': 'manojkumargalla',
    #         'password': '3435isbcmon@G',
    #         'apiUrl': 'https://athena.rc.ufl.edu/api/v1',
    #         'layerName': 'tbm_geojson',
    #         'svsBase': "/blue/pinaki.sarder/nlucarelli/kpmp_new/"
    #     }

    annos = generateAnno(contoursColors)

    # annoooos[name] = annos

#     dsa = DSA(config)

#     fid = dsafid[name]

    # annos = annoooos[name]
    # dsa.postAnno(fid, annos)
    file_name = filename.split('/')[-2]
    json_dir = "/home/manojkumargalla/PostProcess/data/TBM_Annotations/"
    annotations_filename = f'{file_name}_tbm_jul18.json'
    with open(f'{json_dir}/{annotations_filename}', 'w') as outfile:
        json.dump(annos, outfile)
    

processing file - /blue/pinaki.sarder/manojkumargalla/PostProcess/data/WSI_tiles_predictions/S-1905-018731_PAS_2of2/
pred_511_109614_30560_89088x_20480y.png
pred_181_109614_30560_91648x_21504y.png
pred_53_109614_30560_99328x_13824y.png
pred_468_109614_30560_16384x_05632y.png
pred_517_109614_30560_08704x_17920y.png
pred_52_109614_30560_100864x_13312y.png
pred_69_109614_30560_13824x_09216y.png
pred_451_109614_30560_89600x_19968y.png
pred_88_109614_30560_09728x_16896y.png
pred_154_109614_30560_18432x_06656y.png
pred_317_109614_30560_96256x_15360y.png
pred_25_109614_30560_13824x_10752y.png
pred_439_109614_30560_88576x_23552y.png
pred_372_109614_30560_09728x_21504y.png
pred_342_109614_30560_100864x_11776y.png
pred_17_109614_30560_99328x_12288y.png
pred_433_109614_30560_08704x_21504y.png
pred_395_109614_30560_91648x_17920y.png
pred_295_109614_30560_12800x_12800y.png
pred_506_109614_30560_99840x_12800y.png
pred_190_109614_30560_92160x_19968y.png
pred_44_109614_30560_14848x_09216y.png
pred_75_